In [20]:
!pip install torch==2.5.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 20.7 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1
    Uninstalling torch-2.2.1:
      Successfully uninstalled torch-2.2.1


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from copy import deepcopy

/opt/anaconda3/envs/layer_skip/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"Torch Version: {torch.__version__}")

Torch Version: 2.5.1


In [12]:
checkpoint = "facebook/layerskip-llama3.2-1B"
early_exit = 4
device = "cuda" if torch.cuda.is_available() else "cpu"
prompt = "typing import List\ndef bucket_sort(A: List):"

# model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", use_safetensors=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map = {"": device}, use_safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [4]:
model.generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": 128001
}

In [5]:
model.generation_config.__dict__['transformers_version'] = '4.45.0'

In [6]:
model.generation_config.__dict__

{'max_length': 20,
 'max_new_tokens': None,
 'min_length': 0,
 'min_new_tokens': None,
 'early_stopping': False,
 'max_time': None,
 'stop_strings': None,
 'do_sample': False,
 'num_beams': 1,
 'num_beam_groups': 1,
 'penalty_alpha': None,
 'dola_layers': None,
 'use_cache': True,
 'cache_implementation': None,
 'cache_config': None,
 'return_legacy_cache': None,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'min_p': None,
 'typical_p': 1.0,
 'epsilon_cutoff': 0.0,
 'eta_cutoff': 0.0,
 'diversity_penalty': 0.0,
 'repetition_penalty': 1.0,
 'encoder_repetition_penalty': 1.0,
 'length_penalty': 1.0,
 'no_repeat_ngram_size': 0,
 'bad_words_ids': None,
 'force_words_ids': None,
 'renormalize_logits': False,
 'constraints': None,
 'forced_bos_token_id': None,
 'forced_eos_token_id': None,
 'remove_invalid_values': False,
 'exponential_decay_length_penalty': None,
 'suppress_tokens': None,
 'begin_suppress_tokens': None,
 'forced_decoder_ids': None,
 'sequence_bias': None,
 'token_heali

{'max_length': 20, 'max_new_tokens': None, 'min_length': 0, 'min_new_tokens': None, 'early_stopping': False, 'max_time': None, 'stop_strings': None, 'do_sample': False, 'num_beams': 1, 'num_beam_groups': 1, 'penalty_alpha': None, 'dola_layers': None, 'use_cache': True, 'cache_implementation': None, 'cache_config': None, 'return_legacy_cache': None, 'temperature': 1.0, 'top_k': 50, 'top_p': 1.0, 'min_p': None, 'typical_p': 1.0, 'epsilon_cutoff': 0.0, 'eta_cutoff': 0.0, 'diversity_penalty': 0.0, 'repetition_penalty': 1.0, 'encoder_repetition_penalty': 1.0, 'length_penalty': 1.0, 'no_repeat_ngram_size': 0, 'bad_words_ids': None, 'force_words_ids': None, 'renormalize_logits': False, 'constraints': None, 'forced_bos_token_id': None, 'forced_eos_token_id': None, 'remove_invalid_values': False, 'exponential_decay_length_penalty': None, 'suppress_tokens': None, 'begin_suppress_tokens': None, 'forced_decoder_ids': None, 'sequence_bias': None, 'token_healing': False, 'guidance_scale': None, 'low_memory': None, 'watermarking_config': None, 'num_return_sequences': 1, 'output_attentions': False, 'output_hidden_states': False, 'output_scores': False, 'output_logits': None, 'return_dict_in_generate': False, 'pad_token_id': None, 'bos_token_id': 128000, 'eos_token_id': 128001, 'encoder_no_repeat_ngram_size': 0, 'decoder_start_token_id': None, 'is_assistant': False, 'num_assistant_tokens': 20, 'num_assistant_tokens_schedule': 'constant', 'assistant_confidence_threshold': 0.4, 'prompt_lookup_num_tokens': None, 'max_matching_ngram_size': None, 'generation_kwargs': {}, '_from_model_config': True, '_commit_hash': '81deb0f88734409cca506bcefcfb5c6bd2667565', 'transformers_version': '4.46.0.dev0', '_original_object_hash': -2498095420294646743}


In [9]:
weights_memo = {id(w): w for w in model.parameters()}
assistant_model = deepcopy(model, memo=weights_memo) # Clone main model with shared weights
# assistant_model = assistant_model.to(model.device) 
assistant_model.model.layers = assistant_model.model.layers[:early_exit] # Apply early exit
del assistant_model.model.layers[early_exit:]

inputs = tokenizer(prompt, return_tensors="pt").to(device)

# if model.generation_config.pad_token_id is None:
#     model.generation_config.pad_token_id = tokenizer.eos_token_id


outputs = model.generate(**inputs, generation_config=model.generation_config, assistant_model=assistant_model, max_new_tokens=512)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

TypeError: deepcopy() got an unexpected keyword argument 'device_map'

In [2]:
!pip install transformer_lens

  Using cached transformer_lens-2.15.0-py3-none-any.whl.metadata (12 kB)
  Using cached beartype-0.14.1-py3-none-any.whl.metadata (28 kB)
  Using cached better_abc-0.0.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached fancy_einsum-0.0.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-stream-generator-0.0.5.tar.gz (13 kB)
  Preparing metadata (setup.py) ... done
  Using cached wandb-0.19.8-py3-none-macosx_11_0_arm64.whl.metadata (10 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached sentry_sdk-2.22.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using c

In [54]:
import torch
from copy import deepcopy
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformer_lens import HookedTransformer
from transformer_lens.loading_from_pretrained import get_pretrained_model_config
from transformers import AutoConfig



class HookedLayerSkip(HookedTransformer):
    """
    A custom wrapper for the LayerSkip model loaded from a Hugging Face checkpoint.
    It adds early-exit functionality via an assistant model and integrates with the
    llm-transparency-tool by registering hooks for introspection.
    
    Parameters:
      - exit_layer: the index of the layer at which to early exit.
      - num_speculations: number of speculative generations (passed for completeness).
      - checkpoint: the Hugging Face checkpoint name (e.g. "facebook/layerskip-llama3.2-1B").
    """
    def __init__(self, config, exit_layer=None, num_speculations=None, checkpoint=None):
        # Initialize the base HookedTransformer to get introspection capabilities.
        super().__init__(config)
        self.exit_layer = exit_layer
        self.num_speculations = num_speculations
        
        # Load the model from Hugging Face.
        # The checkpoint should be something like "facebook/layerskip-llama3.2-1B"
        self.model = AutoModelForCausalLM.from_pretrained(
            checkpoint,
            device_map="auto",
            use_safetensors=True,
            torch_dtype=torch.bfloat16
        )
        
        # Save the generation config for later use.
        self.generation_config = self.model.generation_config
        self.generation_config.__dict__['transformers_version'] = '4.45.0'

    @classmethod
    def from_pretrained(cls, model_name_or_path, **kwargs):
        """
        Loads a pretrained LayerSkip model (from its Hugging Face checkpoint) and wraps it
        for use with the llm-transparency-tool.
        
        Additional kwargs:
          - exit_layer: for early exit.
          - num_speculations: for speculative generation.
        """
        # config = get_pretrained_model_config(model_name_or_path)
        config = AutoConfig.from_pretrained("facebook/layerskip-llama3.2-1B")
        config = get_pretrained_model_config("meta-llama/Llama-3.2-1B")
        
        
        exit_layer = kwargs.pop("exit_layer", None)
        num_speculations = kwargs.pop("num_speculations", None)
        # Here model_name_or_path is assumed to be the Hugging Face checkpoint name.
        return cls(config=config,exit_layer=exit_layer, num_speculations=num_speculations, checkpoint=model_name_or_path)

    def forward(self, x):
        """
        Runs a forward pass through the model while registering hooks to capture intermediate
        activations. If an exit_layer is specified, hooks are only attached to layers up to that layer.
        """
        # Clear previous activations.
        self.activations = {}

        def save_activation(name):
            def hook(module, input, output):
                self.activations[name] = output
            return hook

        # Locate the transformer submodule.
        # For Llama-based models loaded via Hugging Face, the transformer body is typically in self.model.model.
        if hasattr(self.model, "model"):
            transformer = self.model.model
        else:
            transformer = self.model

        layers = getattr(transformer, "layers", None)
        if layers is None:
            raise ValueError("Could not locate transformer layers for hooking.")
        
        # Register hooks on each layer up to the early exit layer.
        for idx, layer in enumerate(layers):
            if self.exit_layer is not None and idx >= self.exit_layer:
                break
            layer.register_forward_hook(save_activation(f"layer_{idx}"))
        
        # Run the forward pass.
        output = self.model(x)
        return output

    def create_assistant_model(self):
        """
        Creates an assistant model for early exit by deep copying the main model (with shared weights)
        and truncating its transformer layers up to self.exit_layer.
        """
        weights_memo = {id(w): w for w in self.parameters()}
        assistant_model = deepcopy(self, memo=weights_memo)
        # Then truncate the layers as before:
        
        if self.exit_layer is not None:
            if hasattr(assistant_model, "model"):
                assistant_model.model.layers = assistant_model.model.layers[:self.exit_layer]
            else:
                assistant_model.layers = assistant_model.layers[:self.exit_layer]
        return assistant_model


    def generate(self, **kwargs):
        """
        Overrides the generate method. If an early exit is specified, an assistant model is created
        (with truncated layers) and passed via the assistant_model keyword argument.
        """
        if self.exit_layer is not None:
            kwargs["assistant_model"] = self.create_assistant_model()
        return self.model.generate(**kwargs)
    
    def run_with_cache(self, *args, **kwargs):
        """
        Overrides the default run_with_cache behavior to use the assistant (early-exit)
        model if an exit_layer is set.

        This ensures that when you run analysis (via run() which calls run_with_cache)
        on the model, it uses the truncated assistant model and captures activations from
        only the layers up to self.exit_layer.
        """
        if self.exit_layer is not None:
            assistant_model = self.create_assistant_model()
            return assistant_model.run_with_cache(*args, **kwargs)
        else:
            return super().run_with_cache(*args, **kwargs)



In [55]:
lens_model = HookedLayerSkip.from_pretrained(
            "facebook/layerskip-llama3.2-1B",
            # hf_model=hf_model,
            exit_layer=4,         # Set your desired early exit layer.
            num_speculations=6,   # Set your desired number of speculations.
        )

Some parameters are on the meta device because they were offloaded to the disk.


In [59]:
lens_model.to_tokens()

TypeError: HookedTransformer.to_tokens() missing 1 required positional argument: 'input'

In [63]:
# lens_model.run_with_cache()
lens_model.hook_embed

HookPoint()

In [ ]:
HookedTransformerConfig:
{'NTK_by_parts_factor': 32.0,
 'NTK_by_parts_high_freq_factor': 4.0,
 'NTK_by_parts_low_freq_factor': 1.0,
 'act_fn': 'silu',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_scale': np.float64(11.313708498984761),
 'attn_scores_soft_cap': -1.0,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 128,
 'd_mlp': 8192,
 'd_model': 3072,
 'd_vocab': 128256,
 'd_vocab_out': 128256,
 'decoder_start_token_id': None,
 'default_prepend_bos': True,
 'device': device(type='cuda'),
 'dtype': torch.float32,
 'eps': 1e-05,
 'experts_per_token': None,
 'final_rms': True,
 'from_checkpoint': False,
 'gated_mlp': True,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': np.float64(0.014433756729740645),
 'load_in_4bit': False,
 'model_name': 'Llama-3.2-3B',
 'n_ctx': 2048,
 'n_devices': 1,
 'n_heads': 24,
 'n_key_value_heads': 8,
 'n_layers': 28,
 'n_params': 3170893824,
 'normalization_type': 'RMS',
 'num_experts': None,
 'original_architecture': 'LlamaForCausalLM',
 'output_logits_soft_cap': -1.0,
 'parallel_attn_mlp': False,
 'positional_embedding_type': 'rotary',
 'post_embedding_ln': False,
 'relative_attention_max_distance': None,
 'relative_attention_num_buckets': None,
 'rotary_adjacent_pairs': False,
 'rotary_base': 500000.0,
 'rotary_dim': 128,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tie_word_embeddings': False,
 'tokenizer_name': 'meta-llama/Llama-3.2-3B',
 'tokenizer_prepends_bos': None,
 'trust_remote_code': False,
 'ungroup_grouped_query_attention': False,
 'use_NTK_by_parts_rope': True,
 'use_attn_in': False,
 'use_attn_result': False,
 'use_attn_scale': True,
 'use_hook_mlp_in': False,
 'use_hook_tokens': False,
 'use_local_attn': False,
 'use_normalization_before_and_after': False,
 'use_split_qkv_input': False,
 'window_size': None}

In [36]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("facebook/layerskip-llama3.2-1B")
print(config)

LlamaConfig {
  "_name_or_path": "facebook/layerskip-llama3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.0",
  "use_cache": true,
  "vocab_size": 128256
}



In [37]:
print(type(config))

<class 'transformers.models.llama.configuration_llama.LlamaConfig'>


In [18]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("facebook/layerskip-llama3.2-1B")


post_embedding_ln not found
